In [1]:
from biological_fuzzy_logic_networks.DREAM_analysis.utils import (cl_data_to_input, 
                                                                  prepare_cell_line_data, 
                                                                  create_bfz)

import json
import pandas as pd

/dccstor/ipc1/.conda/envs/biofuzznet2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
with open("/dccstor/ipc1/CAR/DREAM/Model/Test/Subnetwork/Configs/0_1_config.json", "rb") as f:
    config = json.load(f)

In [6]:
config.keys()

dict_keys(['pkn_sif', 'network_class', 'data_file', 'output_dir', 'time_point', 'non_marker_cols', 'treatment_col_name', 'minmaxscale', 'add_root_values', 'root_nodes', 'input_value', 'train_treatments', 'valid_treatments', 'train_cell_lines', 'valid_cell_lines', 'convergence_check', 'replace_zero_inputs', 'inhibition_value', 'learning_rate', 'n_epochs', 'batch_size', 'tensors_to_cuda', 'checkpoint_path', 'experiment_name', 'optimizer', 'cell_lines', 'test_cell_lines', 'param_setting'])

In [8]:
config["data_file"]

['/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/BT20.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/BT474.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/BT549.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/CAL148.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/CAL51.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/CAL851.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/DU4475.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/EFM192A.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/EVSAT.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/HBL100.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/HCC1187.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/HCC1395.csv',
 '/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/HCC1419.csv',
 '/dccstor/ipc

In [10]:
cl_data = prepare_cell_line_data(
        data_file          = config["data_file"],
        time_point         = config["time_point"],
        non_marker_cols    = config["non_marker_cols"],
        treatment_col_name = config["treatment_col_name"])


['BT20' 'BT474' 'BT549' 'CAL148' 'CAL51' 'CAL851' 'DU4475' 'EFM192A'
 'EVSAT' 'HBL100' 'HCC1187' 'HCC1395' 'HCC1419' 'HCC1500' 'HCC1569'
 'HCC1599' 'HCC1937' 'HCC1954' 'HCC2185' 'HCC3153' 'HCC38' 'HCC70' 'HDQP1'
 'JIMT1' 'MCF7' 'MDAMB134VI' 'MDAMB157' 'MDAMB175VII' 'MDAMB361'
 'MDAMB415' 'MDAMB453' 'MFM223' 'MPE600' 'MX1' 'OCUBM' 'T47D' 'UACC812'
 'UACC893' 'ZR7530']


In [12]:
model = create_bfz(config["pkn_sif"], config["network_class"])

In [13]:
(
        train_data,
        valid_data,
        train_inhibitors,
        valid_inhibitors,
        train_input,
        valid_input,
        train,
        valid,
        scaler,
    ) = cl_data_to_input(
        data               = cl_data,
        model              = model,
        train_treatments   = config["train_treatments"],
        valid_treatments   = config["valid_treatments"],
        train_cell_lines   = config["train_cell_lines"],
        valid_cell_lines   = config["valid_cell_lines"],
        inhibition_value   = config["inhibition_value"],
        minmaxscale        = config["minmaxscale"],
        add_root_values    = config["add_root_values"],
        input_value        = config["input_value"],
        root_nodes         = config["root_nodes"],
        replace_zero_inputs= config["replace_zero_inputs"],
        balance_data       = True,
    )

Splitting based on validation cell line(s)


/u/adr/Code/biological_fuzzy_logic_networks/biological_fuzzy_logic_networks/DREAM_analysis/utils.py:247: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[markers] = scaler.transform(train[markers])
/u/adr/Code/biological_fuzzy_logic_networks/biological_fuzzy_logic_networks/DREAM_analysis/utils.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid[markers] = scaler.transform(valid[markers])
/u/adr/Code/biological_fuzzy_logic_networks/biological_fuzzy_logic_networks/DREAM_analysis/utils.py:251: Setti

In [22]:
sum(train.groupby(["treatment", "cell_line", "time"]).size() != 500)

0

In [24]:
cl_data = prepare_cell_line_data(
        data_file          = config["test_cell_lines"],
        time_poin          = config["time_point"],
        non_marker_cols    = config["non_marker_cols"],
        treatment_col_name = config["treatment_col_name"],
    )

(test_data, test_inhibitors, test_input, test, scaler) = cl_data_to_input(
    data               =cl_data,
    model              =model,
    train_treatments   =config["train_treatments"],
    valid_treatments   =config["valid_treatments"],
    train_cell_lines   =config["train_cell_lines"],
    valid_cell_lines   =config["valid_cell_lines"],
    inhibition_value   =config["inhibition_value"],
    minmaxscale        =scaler,
    add_root_values    =config["add_root_values"],
    input_value        =config["input_value"],
    root_nodes         =config["root_nodes"],
    do_split           =False,
    replace_zero_inputs=config["replace_zero_inputs"],
    balance_data       =False,
)

['AU565' 'EFM19' 'HCC2218' 'LY2' 'MACLS2' 'MDAMB436']


/u/adr/Code/biological_fuzzy_logic_networks/biological_fuzzy_logic_networks/DREAM_analysis/utils.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t[t < 0] = 0
/u/adr/Code/biological_fuzzy_logic_networks/biological_fuzzy_logic_networks/DREAM_analysis/utils.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t[t < 0] = 0


In [26]:
test.groupby(["treatment", "cell_line", "time"]).size()

treatment  cell_line  time
EGF        AU565      9.0     25647
           EFM19      9.0     18818
           HCC2218    9.0      5315
           LY2        9.0      8963
           MACLS2     9.0     17118
           MDAMB436   9.0     38567
iEGFR      AU565      9.0     10113
           EFM19      9.0     10928
           HCC2218    9.0      2104
           LY2        9.0      5275
           MACLS2     9.0     12531
           MDAMB436   9.0     23418
iMEK       AU565      9.0      9720
           EFM19      9.0      7305
           HCC2218    9.0      2646
           LY2        9.0      3018
           MACLS2     9.0      6105
           MDAMB436   9.0     10374
iPI3K      AU565      9.0     10331
           EFM19      9.0      8536
           HCC2218    9.0      2858
           LY2        9.0      3045
           MACLS2     9.0     12663
           MDAMB436   9.0     16088
iPKC       AU565      9.0      8176
           EFM19      9.0     12693
           HCC2218    9.0      2568
 

In [4]:
import os

In [ ]:
dir_ = "/dccstor/ipc1/CAR/DREAM/DREAMdata/Time_aligned_per_cell_line/CL_incl_test/"
file_list = os.listdir(dir_)
for cl in file_list:
    cl_data = pd.read_csv(os.path.join(dir_, cl))
    cl_data = cl_data[cl_data["time"]==9]
    print(cl, sum(cl_data.groupby(["cell_line", "treatment", "time"]).size()<500))
    

In [ ]:
["HCC1143", "CAMA1", "CAL120", "ZR75B", "KLP1"]

In [6]:
cl_data = pd.read_csv(os.path.join(dir_, "CAMA1.csv"))
cl_data = cl_data[cl_data["time"]==9]
print("CAMA1", sum(cl_data.groupby(["cell_line", "treatment", "time"]).size()<500))

CAMA1 5


In [7]:
cl_data["treatment"].unique()

array(['EGF', 'iEGFR', 'iMEK', 'iPI3K', 'iPKC'], dtype=object)

In [8]:
cl_data

,treatment,cell_line,time,cellID,fileID,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,...,p.PLCg2,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5,time_course
19364,EGF,CAMA1,9.0,NaN,NaN,0.694251,2.004234,1.660828,2.572215,5.394875,...,NaN,3.321990,6.487628,1.668715,1.513456,1.662458,2.774303,1.145099,2.295443,NaN
19373,iEGFR,CAMA1,9.0,NaN,NaN,0.693153,2.181992,1.767259,2.412667,5.308749,...,NaN,3.393616,5.501957,1.705777,1.716499,1.832393,2.881469,1.146622,2.386872,NaN
19380,iMEK,CAMA1,9.0,NaN,NaN,0.731537,2.122155,1.782424,2.530171,5.475270,...,NaN,3.313120,5.525886,1.582884,1.684214,1.776345,2.863942,1.168501,2.390372,NaN
19387,iPI3K,CAMA1,9.0,NaN,NaN,0.649177,2.037379,1.693821,2.433710,5.322280,...,NaN,3.373682,5.521432,1.656749,1.558479,1.691948,2.801347,1.114451,2.303689,NaN
19394,iPKC,CAMA1,9.0,NaN,NaN,0.762445,2.115859,1.774404,2.677387,5.307487,...,NaN,3.271681,5.282989,1.831079,1.726905,1.820804,2.928320,1.228058,2.444894,NaN
